In [1]:
#using sklearn random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from talib import abstract
from ccxt import binance
import pandas as pd
import numpy as np
import os

In [2]:
os.remove("../data/BTC_USDT_15m.csv")

In [3]:
binance = binance()
symbol = 'BTC/USDT'
timeframe = '15m'
file_name = f"../data/{symbol.replace('/', '_')}_{timeframe}.csv"
start = binance.parse8601('2019-10-01T00:00:00Z')
end = binance.parse8601('2022-12-30T00:00:00Z')
cnt_time = start
data = []
while cnt_time < end:
    ohlcv = binance.fetch_ohlcv(symbol, timeframe, cnt_time)
    data += ohlcv
    cnt_time = ohlcv[-1][0] + 900000 # 1h in ms    
df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df['time'] = pd.to_datetime(df['time'], unit='ms')
df.to_csv(file_name, index=False)

In [4]:
df

,time,open,high,low,close,volume
0,2019-10-01 00:00:00,8289.97,8310.08,8206.57,8243.39,1016.103644
1,2019-10-01 00:15:00,8242.94,8252.53,8210.01,8220.39,391.576024
2,2019-10-01 00:30:00,8220.78,8222.13,8188.00,8213.23,285.706881
3,2019-10-01 00:45:00,8213.23,8218.34,8195.36,8199.66,176.788254
4,2019-10-01 01:00:00,8199.21,8222.28,8173.05,8211.85,420.860704
...,...,...,...,...,...,...
113798,2022-12-31 02:45:00,16573.43,16579.79,16569.93,16575.94,771.456450
113799,2022-12-31 03:00:00,16576.18,16577.38,16569.61,16570.76,810.854950
113800,2022-12-31 03:15:00,16570.76,16574.11,16561.49,16565.41,996.310850
113801,2022-12-31 03:30:00,16565.09,16566.00,16552.08,16554.70,1116.508210


In [5]:
df['MFI'] = abstract.MFI(df, timeperiod=14)
df['RSI'] = abstract.RSI(df, timeperiod=14)
df['ADX'] = abstract.ADX(df, timeperiod=14)
df['CCI'] = abstract.CCI(df, timeperiod=14)
df['ATR'] = abstract.ATR(df, timeperiod=14)
df['OBV'] = abstract.OBV(df, timeperiod=14)
df['EMA'] = abstract.EMA(df, timeperiod=14)
df['WILLR'] = abstract.WILLR(df, timeperiod=14)
df['AD'] = abstract.AD(df, timeperiod=14)
df['ADOSC'] = abstract.ADOSC(df, timeperiod=14)
df['ADXR'] = abstract.ADXR(df, timeperiod=14)
df['APO'] = abstract.APO(df, timeperiod=14)
df['AROONOSC'] = abstract.AROONOSC(df, timeperiod=14)
df['BOP'] = abstract.BOP(df, timeperiod=14)
df['CCI'] = abstract.CCI(df, timeperiod=14)
df['CMO'] = abstract.CMO(df, timeperiod=14)
df['DX'] = abstract.DX(df, timeperiod=14)
df['MOM'] = abstract.MOM(df, timeperiod=14)
df['PPO'] = abstract.PPO(df, timeperiod=14)
df['ROC'] = abstract.ROC(df, timeperiod=14)
df['ROCP'] = abstract.ROCP(df, timeperiod=14)
df['ROCR'] = abstract.ROCR(df, timeperiod=14)
df['ROCR100'] = abstract.ROCR100(df, timeperiod=14)
df['RSI'] = abstract.RSI(df, timeperiod=14)
df['TRIX'] = abstract.TRIX(df, timeperiod=14)
df['ULTOSC'] = abstract.ULTOSC(df, timeperiod=14)
df['WILLR'] = abstract.WILLR(df, timeperiod=14)
df['WMA'] = abstract.WMA(df, timeperiod=14)
df['HT_TRENDLINE'] = abstract.HT_TRENDLINE(df, timeperiod=14)
df['TRANGE'] = abstract.TRANGE(df, timeperiod=14)
df['CLOSE_percent'] = df['close'].pct_change()

In [6]:
df['UP'] = df['CLOSE_percent'].apply(lambda x: 1 if x > 0 else 0)
df['DOWN'] = df['CLOSE_percent'].apply(lambda x: 1 if x < 0 else 0)
df['UP'] = df['UP'].shift(-1) #shift UP DOWN 一個單位，因為我們要預測的是下一個時間點的漲跌
df['DOWN'] = df['DOWN'].shift(-1)

df = df.dropna()

In [7]:
#正規化
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['MFI','RSI','ADX','CCI','ATR','OBV','EMA','WILLR','AD','ADOSC','ADXR','APO','AROONOSC','BOP','CCI','CMO','DX','MOM','PPO','ROC','ROCP','ROCR','ROCR100','RSI','TRIX','ULTOSC','WILLR','WMA','HT_TRENDLINE','TRANGE']] = scaler.fit_transform(df[['MFI','RSI','ADX','CCI','ATR','OBV','EMA','WILLR','AD','ADOSC','ADXR','APO','AROONOSC','BOP','CCI','CMO','DX','MOM','PPO','ROC','ROCP','ROCR','ROCR100','RSI','TRIX','ULTOSC','WILLR','WMA','HT_TRENDLINE','TRANGE']])
df[['open','high','low','close','volume']] = scaler.fit_transform(df[['open','high','low','close','volume']])

C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [8]:
X = df.drop(['UP','DOWN','time','CLOSE_percent'], axis=1)
#y -> UP and DOWN
y = df['UP'] #UP

In [9]:
X

,open,high,low,close,volume,MFI,RSI,ADX,CCI,ATR,...,PPO,ROC,ROCP,ROCR,ROCR100,TRIX,ULTOSC,WMA,HT_TRENDLINE,TRANGE
63,0.068547,0.063779,0.070296,0.069147,0.009006,0.545123,0.573676,0.110529,0.663657,0.017247,...,0.665069,0.525270,0.525270,0.525270,0.525270,0.652433,0.483833,0.060348,0.056874,0.005730
64,0.069061,0.063711,0.070232,0.068538,0.005545,0.583632,0.478211,0.097099,0.543233,0.017270,...,0.663999,0.521721,0.521721,0.521721,0.521721,0.652979,0.423186,0.060335,0.056875,0.005697
65,0.068445,0.063783,0.070170,0.069236,0.006517,0.645869,0.569353,0.087159,0.630719,0.017612,...,0.664184,0.530577,0.530577,0.530577,0.530577,0.654181,0.571219,0.060415,0.056885,0.006910
66,0.069142,0.065076,0.070752,0.070208,0.028755,0.666238,0.661885,0.109944,0.828155,0.019634,...,0.666379,0.536803,0.536803,0.536803,0.536803,0.656909,0.610421,0.060617,0.056919,0.013368
67,0.070112,0.065230,0.071492,0.070131,0.014693,0.677433,0.650392,0.133758,0.750029,0.020111,...,0.668514,0.536073,0.536073,0.536073,0.536073,0.660414,0.595304,0.060796,0.056964,0.008062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113797,0.195580,0.190465,0.197587,0.195735,0.022960,0.436237,0.499643,0.160431,0.437095,0.006538,...,0.662161,0.517863,0.517863,0.517863,0.517863,0.662850,0.428888,0.189066,0.186483,0.001335
113798,0.195661,0.190537,0.197654,0.195780,0.019109,0.430939,0.514947,0.152548,0.463577,0.006189,...,0.660823,0.517987,0.517987,0.517987,0.517987,0.662195,0.500045,0.189051,0.186466,0.001382
113799,0.195704,0.190500,0.197649,0.195700,0.020085,0.347936,0.486486,0.144850,0.456942,0.005786,...,0.659909,0.515605,0.515605,0.515605,0.515605,0.661598,0.492841,0.189028,0.186446,0.001089
113800,0.195620,0.190449,0.197524,0.195618,0.024679,0.274824,0.458040,0.129274,0.424947,0.005592,...,0.659197,0.515861,0.515861,0.515861,0.515861,0.661014,0.465334,0.189001,0.186427,0.001769


In [10]:
y.value_counts()

1.0    57220
0.0    56519
Name: UP, dtype: int64

In [11]:
X = X.values
y = y.values

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [13]:
X_train

array([[0.07401561, 0.06894444, 0.07571084, ..., 0.06612936, 0.06294052,
        0.00351595],
       [0.24990375, 0.24593722, 0.25189686, ..., 0.24413769, 0.24185866,
        0.01235629],
       [0.28273195, 0.2782367 , 0.28370795, ..., 0.27735791, 0.27621881,
        0.01706385],
       ...,
       [0.04843619, 0.04305331, 0.04948957, ..., 0.04031012, 0.03686429,
        0.00628749],
       [0.65859559, 0.65750067, 0.66158932, ..., 0.65500844, 0.65230799,
        0.03286739],
       [0.27439417, 0.27277663, 0.27627633, ..., 0.27078841, 0.26937049,
        0.03489873]])

In [14]:
#random forest classifier with n_estimators=10 (default)
model = RandomForestClassifier(
    n_estimators=120)



In [15]:
#train the model using the training sets y_pred=clf.predict(X_test)
model.fit(X_train,y_train)

RandomForestClassifier(n_estimators=120)

In [16]:
pred = model.predict(X_test)

print(accuracy_score(y_test, pred))

0.5304378406892913


In [17]:
pred

array([0., 1., 1., ..., 1., 1., 0.])

In [18]:
X_train[0]

array([0.07401561, 0.06894444, 0.07571084, 0.07419365, 0.00975678,
       0.41563856, 0.42356651, 0.08950581, 0.44580353, 0.01745007,
       0.24720223, 0.06370615, 0.47859029, 0.09165883, 0.51551069,
       0.13764638, 0.55694102, 0.42857143, 0.61144338, 0.42356651,
       0.15582398, 0.5204113 , 0.65045235, 0.51442521, 0.51442521,
       0.51442521, 0.51442521, 0.63300074, 0.5385446 , 0.06612936,
       0.06294052, 0.00351595])

In [19]:
test_ans = model.predict(X_train)
print(accuracy_score(y_train, test_ans))

1.0


In [20]:
#confusion matrix
confusion_matrix(y_test, pred)


array([[7566, 6765],
       [6587, 7517]], dtype=int64)

In [21]:
#classification report  
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       0.53      0.53      0.53     14331
         1.0       0.53      0.53      0.53     14104

    accuracy                           0.53     28435
   macro avg       0.53      0.53      0.53     28435
weighted avg       0.53      0.53      0.53     28435



In [22]:
#ROC curve
fpr, tpr, thresholds = roc_curve(y_test, pred)
auc = roc_auc_score(y_test, pred)
print('AUC: %.2f' % auc)

AUC: 0.53
